# NES-MDB

The purpose of this notebook is to showcase a few things about the NES MDB and our work on it.

In [2]:
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import pickle

The purpose of this notebook is to showcase a few things about the NES MDB and our work on it.

# 1. Music Statistics.

We begin by first loading the data. We apply a preprocessing step where we move all the notes so that there is no spacing between 0 and the rest of the integers. We remark that for P1 and P2, the note 32 is never played, so we ignore it and shift it down.

In [4]:
def song_loader(folder):
    processed_songs = []

    #instrument info.
    #(key,val) = (instrument, (note_size,velocity_size, timbre_size))
    voices = {"P1": (77,16,4),
                   "P2": (77,16,4),
                   "TR": (89,0,0),
                   "NO": (17,16,2)}

    #First, we define the functions which will normalize our notes
    #to begin at 0.

    P1_normalizer = np.vectorize(lambda x : x - 32 if x > 0 else 0)
    P2_normalizer = np.vectorize(lambda x : x - 32 if x > 0 else 0)
    TR_normalizer = np.vectorize(lambda x : x - 20 if x > 0 else 0)



    for song in glob.glob(folder + '/*'):
        with open(song, 'rb') as song_info:
            rate, nsamps, exprsco = pickle.load(song_info)

            #Normalize the notes for the voices.
            exprsco[:,0] = P1_normalizer(exprsco[:,0])
            exprsco[:,1] = P2_normalizer(exprsco[:,1])
            exprsco[:,2] = TR_normalizer(exprsco[:,2])

            processed_songs.append(exprsco)
    
    return processed_songs

def note_counter(songs):
    P1 = Counter()
    P2 = Counter()
    TR = Counter()
    NO = Counter()
    voice_counts = [P1,P2,TR,NO]

    for song in processed_songs:
        for voice in range(4):
            for note in song[:,voice]:
                voice_counts[voice][note] += 1
    
    return voice_counts

processed_songs   = song_loader(folder = 'nesmdb24_seprsco/train')
voice_counts = note_counter(processed_songs)

KeyboardInterrupt: 

We begin by first preparing a function to count the number of times each notes appears for each instrument. As it turns out, the most popular note is "0" i.e. that the instrument is not playing. We will plot the counts for the all the notes per instrument in a moment, but we first want to emphasize just how often each voice is turned off. Most notably, notice that for the noise voice, 0 appears more than 50% of the time. It might even be worth considering removing it entirely for the dimension reduction. 

In [ ]:
voices = ["P1","P2","TR","NO"]
tot_count = sum(voice_counts[0].values())

for voice,note_count in zip(voices,voice_counts):
    percentage = 100*note_count[0]/tot_count
    print(voice + " is turned off for {:.2f}% of the time.".format(percentage))

In [ ]:
def note_plotter(ax,voice_counts):
    notes,counts = [],[]
    for note, count in voice_counts.items():
        notes.append(note)
        counts.append(count)
    max_note = max(notes)+1
    #plt.figure(figsize=(20,10))
    #plt.xticks(np.arange(0, max_note, 1))
    ax.hist(notes, weights= counts, bins=range(max_note))
    ax.set_xlim(0,max_note-1)

fig, (ax1, ax2) = plt.subplots(1,2, figsize = (10,7))
fig.suptitle("Histograms for Pulses",fontsize = 40)

#First, we do voice P1.
note_plotter(ax1,voice_counts[0])
ax1.set_xlabel("Notes for P1", fontsize=20)
ax1.set_ylabel("Counts", fontsize=20)
ax1.set_ylim(0,600000)

#Now voice P2
note_plotter(ax2,voice_counts[1])
ax2.set_xlabel("Notes for P2", fontsize=20)
ax2.set_ylim(0,600000)
fig.tight_layout()
fig.subplots_adjust(top=0.88)

In [ ]:
fig, ax = plt.subplots(figsize=(11,7))
note_plotter(ax, voice_counts[2])
ax.set_xlabel("Notes for TR", fontsize=20)
ax.set_ylabel("Counts", fontsize=20)
_ = ax.set_title("Histogram for TR voice", fontsize=20)

In [ ]:
fig, ax = plt.subplots(figsize=(11,7))
note_plotter(ax, voice_counts[3])
ax.set_xlabel("Notes for NO", fontsize=20)
ax.set_ylabel("Counts", fontsize=20)
_ = ax.set_title("Histogram for NO voice", fontsize=20)

## Frozen time

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
j = 2
voice_names = ["P1","P2", "TR","NO"]
fig, ax = plt.subplots(figsize = (17,7))
sns.barplot(x= [i for i in range(len(cnts[j]))], y = [len(Z) for Z in cnts[j]], ax = ax)
ax.set_xlabel("Note range", fontsize = 20)
ax.set_ylabel("Count", fontsize = 20)
_ = ax.set_title("How many times did each " + voice_names[j] +" note get held?", fontsize=20)

In [ ]:
fig2, ax2 = plt.subplots(figsize = (15,7))
sns.countplot(x=cnts[3][0], ax = ax2)
ax2.set_title("Number of times the NO voice was turned off per duration.", fontsize=20)
ax2.set_xlabel("Number of timesteps held.", fontsize= 20)
ax2.set_ylabel("Count", fontsize=20)

In [ ]:
fig2, ax2 = plt.subplots(figsize = (15,7))
sns.countplot(x=cnts[2][0], ax = ax2)
ax2.set_title("Number of times the TR voice was turned off per duration.", fontsize=20)
ax2.set_xlabel("Number of timesteps held.", fontsize= 20)
ax2.set_ylabel("Count", fontsize=20)

In [ ]:
fig2, ax2 = plt.subplots(figsize = (15,7))
sns.countplot(x=cnts[1][0], ax = ax2)
ax2.set_title("Number of times the P2 voice was turned off per duration.", fontsize=20)
ax2.set_xlabel("Number of timesteps held.", fontsize= 20)
ax2.set_ylabel("Count", fontsize=20)

In [ ]:
fig2, ax2 = plt.subplots(figsize = (15,7))
sns.countplot(x=cnts[0][0], ax = ax2)
ax2.set_title("Number of times the P1 voice was turned off per duration.", fontsize=20)
ax2.set_xlabel("Number of timesteps held.", fontsize= 20)
ax2.set_ylabel("Count", fontsize=20)

## 2. Song Dictionary.

During our preprocessing, we chopped the music into several pieces. In order to assign meaning to each piece, we created a dictionary which takes a song name and gives you all the pieces associated with that name. We leave it for reference.

In [3]:
with open('52_12_TR_song_dict','rb') as file:
    song_dict = pickle.load(file)

In [24]:
#song_dict['nesmdb24_seprsco/train/380_WaronWheels_10_11ResultsTheme.seprsco.pkl']
#song_dict['nesmdb24_seprsco/train/221_MarioBros__01_02GameStartA.seprsco.pkl']
#song_dict['nesmdb24_seprsco/train/303_SpaceHarrier_11_12Hayaoh.seprsco.pkl']
#song_dict['nesmdb24_seprsco/train/346_TheJungleBook_04_05Level4Level7Level10.seprsco.pkl']
#song_dict['nesmdb24_seprsco/train/199_Labyrinth_MaounoMeikyu_08_09Oubliette.seprsco.pkl']
#song_dict['nesmdb24_seprsco/train/082_Dr_Mario_08_09Ending.seprsco.pkl']
#song_dict['nesmdb24_seprsco/train/105_FamicomJump_HeroRetsuden_06_07AdventureJumpWorld.seprsco.pkl']
#song_dict['nesmdb24_seprsco/train/117_FinalFantasy_11_12MenuScreen.seprsco.pkl']
#song_dict['nesmdb24_seprsco/train/053_ChoujinSentaiJetman_01_02AreaSelect.seprsco.pkl']
#song_dict['nesmdb24_seprsco/train/329_SwordMaster_06_07Stage3.seprsco.pkl']
#song_dict['nesmdb24_seprsco/train/025_BatmanReturns_03_04Opening1.seprsco.pkl']
#song_dict['nesmdb24_seprsco/train/032_BioSenshiDan_IncreasertonoTatakai_04_05Stage3.seprsco.pkl']
song_dict['nesmdb24_seprsco/train/322_SuperMarioBros__02_03SwimmingAround.seprsco.pkl']

[1569226,
 1569227,
 1569228,
 1569229,
 1569230,
 1569231,
 1569232,
 1569233,
 1569234,
 1569235,
 1569236,
 1569237,
 1569238,
 1569239,
 1569240,
 1569241,
 1569242,
 1569243,
 1569244,
 1569245,
 1569246,
 1569247,
 1569248,
 1569249,
 1569250,
 1569251,
 1569252,
 1569253,
 1569254,
 1569255,
 1569256,
 1569257,
 1569258,
 1569259,
 1569260,
 1569261,
 1569262,
 1569263,
 1569264,
 1569265,
 1569266,
 1569267,
 1569268,
 1569269,
 1569270,
 1569271,
 1569272,
 1569273,
 1569274,
 1569275,
 1569276,
 1569277,
 1569278,
 1569279,
 1569280,
 1569281,
 1569282,
 1569283,
 1569284,
 1569285,
 1569286,
 1569287,
 1569288,
 1569289,
 1569290]

In [23]:
song_dict['nesmdb24_seprsco/train/325_SuperMarioWorld_01_02YoshisIsland.seprsco.pkl']

[591037,
 591038,
 591039,
 591040,
 591041,
 591042,
 591043,
 591044,
 591045,
 591046,
 591047,
 591048,
 591049,
 591050,
 591051,
 591052,
 591053]

In [ ]:
song_dic